In [1]:
import numpy as np
import pandas as pd

from datetime import datetime
from time import sleep

import pickle

from pprint import pprint

from hab.airtable.airtable import AirtableHAB
from hab.serpapi import serpapi, locations
from hab.clean_functions import clean

In [2]:
today = datetime.now().strftime("%Y%m%d")
# print(today)

with open(file = "serpapi_key.txt", mode = "r") as file:
    api_key = file.read()
    
# print(api_key)

lista_puestos = pd.read_excel("Lista_Puestos.xlsx", header = None)

lista_puestos.columns = ["jobs"]

In [3]:
lista_puestos

,jobs
0,Desarrollador de software
1,Programador web
2,Consultor tecnológico
3,Ingeniero de sistemas
4,Especialista en seguridad informática
...,...
478,Desarrollador de software científico
479,Técnico de soporte remoto
480,Especialista en redes informáticas
481,Analista de seguridad en la nube


In [4]:
with open("lista_puestos_ingles.txt", "r") as file:
    lista_puestos_ingles = file.read()
    
lista_puestos_ingles = lista_puestos_ingles.split("\n")

lista_puestos_ingles

['Software Developer',
 'Web Programmer',
 'Technology Consultant',
 'Systems Engineer',
 'Information Security Specialist',
 'Programmer Analyst',
 'Mobile App Developer',
 'Database Administrator',
 'Software Architect',
 'Network Technician',
 'Cybersecurity Specialist',
 'IT Project Manager',
 'UX/UI Designer',
 'DevOps Engineer',
 'Support Technician',
 'Data Analyst',
 'Artificial Intelligence Specialist',
 'Video Game Developer',
 'QA Tester',
 'SAP Consultant',
 'Full-stack Developer',
 'Digital Graphic Designer',
 'Blockchain Specialist',
 'Data Scientist',
 'Software Engineer',
 'Big Data Consultant',
 'Systems Administrator',
 'Cloud Computing Specialist',
 'Information Security Analyst',
 'Java Developer',
 'Systems Technician',
 'Machine Learning Specialist',
 'Front-end Developer',
 'IoT Specialist (Internet of Things)',
 'Digital Transformation Consultant',
 '.NET Developer',
 'Virtual Reality Specialist',
 'Industrial Cybersecurity Specialist',
 'Telecommunications Tech

In [5]:
[x for x in [x.replace("Developer", "") for x in lista_puestos_ingles] if len(x.split()) > 1]

['Web Programmer',
 'Technology Consultant',
 'Systems Engineer',
 'Information Security Specialist',
 'Programmer Analyst',
 'Mobile App ',
 'Database Administrator',
 'Software Architect',
 'Network Technician',
 'Cybersecurity Specialist',
 'IT Project Manager',
 'UX/UI Designer',
 'DevOps Engineer',
 'Support Technician',
 'Data Analyst',
 'Artificial Intelligence Specialist',
 'Video Game ',
 'QA Tester',
 'SAP Consultant',
 'Digital Graphic Designer',
 'Blockchain Specialist',
 'Data Scientist',
 'Software Engineer',
 'Big Data Consultant',
 'Systems Administrator',
 'Cloud Computing Specialist',
 'Information Security Analyst',
 'Systems Technician',
 'Machine Learning Specialist',
 'IoT Specialist (Internet of Things)',
 'Digital Transformation Consultant',
 'Virtual Reality Specialist',
 'Industrial Cybersecurity Specialist',
 'Telecommunications Technician',
 'Security Consultant',
 'Deep Learning Specialist',
 'Data Mining Specialist',
 'Network Specialist',
 'Enterprise Arc

In [6]:
latam = [["com.ar", "Argentina"]         ,
         ["com.bo", "Bolivia"]           ,
         ["com.br", "Brazil"]            ,
         ["ca", "Canada"]            ,
         ["cl", "Chile"]             ,
         ["com.co", "Colombia"]          ,
         ["co.cr", "Costa Rica"]        ,
         ["com.cu", "Cuba"]              ,
         ["com.do", "Dominican Republic"],
         ["com.ec", "Ecuador"]           ,
         ["com.sv", "El Salvador"]       ,
         ["com.gt", "Guatemala"]         ,
#          ["gy", "Guyana"]            ,
#          ["ht", "Haiti"]             ,
         ["hn", "Honduras"]          ,
#          ["jm", "Jamaica"]           ,
         ["com.mx", "Mexico"]            ,
         ["com.ni", "Nicaragua"]         ,
         ["com.pa", "Panama"]            ,
         ["com.py", "Paraguay"]          ,
         ["com.pe", "Peru"]              ,
         ["com.pr", "Puerto Rico"]       ,
         ["com.uy", "Uruguay"]           ,
#          ["ve", "Venezuela"]
        ]

latam = [[".es", "Spain"]]

latam

[['.es', 'Spain']]

In [7]:
lista_puestos["jobs"].values

array(['Desarrollador de software', ' Programador web',
       ' Consultor tecnológico', ' Ingeniero de sistemas',
       ' Especialista en seguridad informática', ' Analista programador',
       ' Desarrollador de aplicaciones móviles',
       ' Administrador de bases de datos', ' Arquitecto de software',
       ' Técnico de redes', ' Especialista en ciberseguridad',
       ' Project manager IT', ' Diseñador UX/UI', ' DevOps',
       ' Técnico de soporte', ' Analista de datos',
       ' Especialista en inteligencia artificial',
       ' Desarrollador de videojuegos', ' QA Tester', ' Consultor SAP',
       'Desarrollador full-stack', ' Diseñador gráfico digital',
       ' Especialista en blockchain', ' Científico de datos',
       ' Ingeniero de software', ' Consultor Big Data',
       ' Administrador de sistemas', ' Especialista en cloud computing',
       ' Analista de seguridad informática', ' Desarrollador Java',
       ' Técnico de sistemas', ' Especialista en machine learning',
 

In [8]:
%%time

df = pd.DataFrame()

for abr, country in latam:
    
    for q in lista_puestos["jobs"].values:

        try:
            for pagination in range(100):
                
#                 sleep(0.3)

                print(f"{q:60}{pagination}")

                q_params = {"q"             : q,
                            "api_key"       : api_key,
                            "location"      : country.title(),
                            "start"         : pagination*10,
#                             "gl"            : abr,
#                             "hl"            : "es",
#                             "google_domain" : f"google.{abr}",
#                             "chips"         : "date_posted:year"
                            }

                response = serpapi.job_search(**q_params)


                if ("error" in response) or (response.get("jobs_results") == None) or (len(response.get("jobs_results")) < 10):
                    break
                
                now = datetime.now()
                
                with open(file = f"json_searches_latam/{str(now.timestamp()).replace('.', '')}.pkl", mode = "bw") as file:
                    pickle.dump(response, file)

                df_response = pd.json_normalize(response["jobs_results"])

                df_response.columns = [x.split(".")[0] if len(x.split(".")) == 1 else x.split(".")[-1] for x in df_response.columns]

                df_response["country_search"] = country.title()
            #     df_response = df_response[["job_id", "title", "company_name", "location", "source", "description", "country", "date_posted", "contract_type", "remote_work"]]

                df = pd.concat(objs = [df, df_response], ignore_index = True)
        except:
            print(f"Error {q} ***************************************************************")
            
        
            
            
df = df.drop_duplicates(subset = "job_id").reset_index(drop = True)

df.to_csv(f"extraccion_{today}_españa_español.csv", index = False)
df.to_pickle(f"extraccion_{today}_españa_español.pkl")

Desarrollador de software                                   0
https://serpapi.com/search
Desarrollador de software                                   1
https://serpapi.com/search
Desarrollador de software                                   2
https://serpapi.com/search
Desarrollador de software                                   3
https://serpapi.com/search
Desarrollador de software                                   4
https://serpapi.com/search
Desarrollador de software                                   5
https://serpapi.com/search
Desarrollador de software                                   6
https://serpapi.com/search
Desarrollador de software                                   7
https://serpapi.com/search
Desarrollador de software                                   8
https://serpapi.com/search
Desarrollador de software                                   9
https://serpapi.com/search
Desarrollador de software                                   10
https://serpapi.com/search
Desarrollador de sof

 Administrador de bases de datos                            10
https://serpapi.com/search
 Administrador de bases de datos                            11
https://serpapi.com/search
 Arquitecto de software                                     0
https://serpapi.com/search
 Arquitecto de software                                     1
https://serpapi.com/search
 Arquitecto de software                                     2
https://serpapi.com/search
 Arquitecto de software                                     3
https://serpapi.com/search
 Arquitecto de software                                     4
https://serpapi.com/search
 Arquitecto de software                                     5
https://serpapi.com/search
 Arquitecto de software                                     6
https://serpapi.com/search
 Arquitecto de software                                     7
https://serpapi.com/search
 Arquitecto de software                                     8
https://serpapi.com/search
 Arquitecto de soft

 Técnico de soporte                                         3
https://serpapi.com/search
 Técnico de soporte                                         4
https://serpapi.com/search
 Técnico de soporte                                         5
https://serpapi.com/search
 Técnico de soporte                                         6
https://serpapi.com/search
 Técnico de soporte                                         7
https://serpapi.com/search
 Técnico de soporte                                         8
https://serpapi.com/search
 Técnico de soporte                                         9
https://serpapi.com/search
 Técnico de soporte                                         10
https://serpapi.com/search
 Técnico de soporte                                         11
https://serpapi.com/search
 Técnico de soporte                                         12
https://serpapi.com/search
 Técnico de soporte                                         13
https://serpapi.com/search
 Técnico de sopor

 Ingeniero de software                                      8
https://serpapi.com/search
 Ingeniero de software                                      9
https://serpapi.com/search
 Ingeniero de software                                      10
https://serpapi.com/search
 Ingeniero de software                                      11
https://serpapi.com/search
 Ingeniero de software                                      12
https://serpapi.com/search
 Ingeniero de software                                      13
https://serpapi.com/search
 Ingeniero de software                                      14
https://serpapi.com/search
 Ingeniero de software                                      15
https://serpapi.com/search
 Ingeniero de software                                      16
https://serpapi.com/search
 Ingeniero de software                                      17
https://serpapi.com/search
 Ingeniero de software                                      18
https://serpapi.com/search
 Ingeniero d

 Desarrollador de Front-end                                 21
https://serpapi.com/search
 Desarrollador de Front-end                                 22
https://serpapi.com/search
 Desarrollador de Front-end                                 23
https://serpapi.com/search
 Desarrollador de Front-end                                 24
https://serpapi.com/search
 Desarrollador de Front-end                                 25
https://serpapi.com/search
 Especialista en IoT                                        0
https://serpapi.com/search
 Consultor de transformación digital                        0
https://serpapi.com/search
 Consultor de transformación digital                        1
https://serpapi.com/search
 Consultor de transformación digital                        2
https://serpapi.com/search
 Consultor de transformación digital                        3
https://serpapi.com/search
 Desarrollador .NET                                         0
https://serpapi.com/search
 Desarrollador .

...

 Especialista en machine learning en el cloud               0
https://serpapi.com/search
 Especialista en sistemas de gestión de contenidos          0
https://serpapi.com/search
 Especialista en inteligencia de negocios en el cloud       0
https://serpapi.com/search
 Especialista en ciberseguridad de aplicaciones             0
https://serpapi.com/search
 Especialista en ciberseguridad de aplicaciones             1
https://serpapi.com/search
 Especialista en ciberseguridad de aplicaciones             2
https://serpapi.com/search
 Especialista en ciberseguridad de aplicaciones             3
https://serpapi.com/search
 Especialista en ciberseguridad de aplicaciones             4
https://serpapi.com/search
 Especialista en ciberseguridad de aplicaciones             5
https://serpapi.com/search
 Especialista en administración de servidores               0
https://serpapi.com/search
 Especialista en implementación de ERP.                     0
https://serpapi.com/search
Especialista en segur

 Programador de Android                                     8
https://serpapi.com/search
 Programador de Android                                     9
https://serpapi.com/search
 Programador de Android                                     10
https://serpapi.com/search
 Programador de iOS                                         0
https://serpapi.com/search
 Programador de iOS                                         1
https://serpapi.com/search
 Programador de iOS                                         2
https://serpapi.com/search
 Programador de iOS                                         3
https://serpapi.com/search
 Programador de iOS                                         4
https://serpapi.com/search
 Programador de iOS                                         5
https://serpapi.com/search
 Programador de iOS                                         6
https://serpapi.com/search
 Desarrollador de sistemas de audio                         0
https://serpapi.com/search
 Desarrollador de si

 Técnico de soporte a usuarios                              4
https://serpapi.com/search
 Técnico de soporte de hardware                             0
https://serpapi.com/search
 Técnico de soporte de hardware                             1
https://serpapi.com/search
 Desarrollador de soluciones para comercio electrónico      0
https://serpapi.com/search
 Especialista en análisis forense digital                   0
https://serpapi.com/search
 Especialista en tecnología VoIP                            0
https://serpapi.com/search
 Especialista en automatización industrial                  0
https://serpapi.com/search
 Especialista en automatización industrial                  1
https://serpapi.com/search
 Técnico en virtualización de sistemas                      0
https://serpapi.com/search
 Técnico en virtualización de sistemas                      1
https://serpapi.com/search
 Desarrollador de sistemas para la industria farmacéutica   0
https://serpapi.com/search
 Técnico en gestión d

 Desarrollador Ruby                                         1
https://serpapi.com/search
 Desarrollador Ruby                                         2
https://serpapi.com/search
 Especialista en microcontroladores                         0
https://serpapi.com/search
Desarrollador Node.js                                       0
https://serpapi.com/search
Desarrollador Node.js                                       1
https://serpapi.com/search
Desarrollador Node.js                                       2
https://serpapi.com/search
Desarrollador Node.js                                       3
https://serpapi.com/search
Desarrollador Node.js                                       4
https://serpapi.com/search
Desarrollador Node.js                                       5
https://serpapi.com/search
Desarrollador Node.js                                       6
https://serpapi.com/search
Desarrollador Node.js                                       7
https://serpapi.com/search
Desarrollador Node.js

Desarrollador Angular                                       5
https://serpapi.com/search
Desarrollador Angular                                       6
https://serpapi.com/search
Desarrollador Angular                                       7
https://serpapi.com/search
Desarrollador Angular                                       8
https://serpapi.com/search
Desarrollador Angular                                       9
https://serpapi.com/search
Desarrollador Angular                                       10
https://serpapi.com/search
Desarrollador Angular                                       11
https://serpapi.com/search
Desarrollador Angular                                       12
https://serpapi.com/search
Desarrollador Angular                                       13
https://serpapi.com/search
Desarrollador Angular                                       14
https://serpapi.com/search
 Analista de datos de recursos humanos                      0
https://serpapi.com/search
 Analista de dat

In [9]:
response

{'search_metadata': {'id': '64f4fdf22bc7d84ef8349dda',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/efa9e1025ccb6368/64f4fdf22bc7d84ef8349dda.json',
  'created_at': '2023-09-03 21:43:14 UTC',
  'processed_at': '2023-09-03 21:43:14 UTC',
  'google_jobs_url': 'https://www.google.com/search?q=+Desarrollador+de+software+educativo&ibp=htl;jobs&uule=w+CAIQICIFU3BhaW4&start=0',
  'raw_html_file': 'https://serpapi.com/searches/efa9e1025ccb6368/64f4fdf22bc7d84ef8349dda.html',
  'total_time_taken': 1.91},
 'search_parameters': {'q': ' Desarrollador de software educativo',
  'engine': 'google_jobs',
  'location_requested': 'Spain',
  'location_used': 'Spain',
  'google_domain': 'google.com',
  'start': 0},
 'search_information': {'jobs_results_state': 'Fully empty'},
 'error': "Google hasn't returned any results for this query."}

In [10]:
df_new = df.copy()
df_new.drop_duplicates(subset = "description", inplace = True)
df_new.reset_index(drop = True, inplace = True)

df_new.shape

(9943, 15)

In [11]:
df_new["date_posted"] = datetime.strptime("2023-08-02", "%Y-%m-%d").date()
df_new

,title,company_name,location,via,description,job_highlights,related_links,thumbnail,extensions,job_id,posted_at,schedule_type,country_search,work_from_home,salary,date_posted
0,Desarrollador de software Full Stack (JavaScri...,etalentum Selección,"La Garriga, Spain",via LinkedIn,Empresa de consolidada trayectoria y en consta...,[{'items': ['Empresa de consolidada trayectori...,[{'link': 'https://www.google.com/search?sca_e...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[3 days ago, Full-time]",eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yIGRlIHNvZn...,3 days ago,Full-time,Spain,NaN,NaN,2023-08-02
1,desarrollador de software,Comunitea Servicios Tecnológicos S.L.,"Lugo, Spain",via Trabajo.org,Buscamos un desarrollador de software para inc...,[{'items': ['Buscamos un desarrollador de soft...,[{'link': 'https://www.google.com/search?sca_e...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[1 day ago, Full-time, No degree mentioned]",eyJqb2JfdGl0bGUiOiJkZXNhcnJvbGxhZG9yIGRlIHNvZn...,1 day ago,Full-time,Spain,NaN,NaN,2023-08-02
2,Desarrollador de software en Java- ARANJUEZ/ T...,Krell Consulting,"Madrid, Spain",via LinkedIn,Profesional con experiencia en desarrollo SW e...,[{'items': ['Profesional con experiencia en de...,[{'link': 'https://www.google.com/search?sca_e...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[5 days ago, Full-time, No degree mentioned]",eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yIGRlIHNvZn...,5 days ago,Full-time,Spain,NaN,NaN,2023-08-02
3,Desarrollador de front-end,WeAreHiring,"Barcelona, Spain",via LinkedIn,Company: Gartner\n\nPosition: Senior Frontend ...,[{'items': ['Company: Gartner Position: Senio...,[{'link': 'https://www.google.com/search?sca_e...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[13 hours ago, Full-time]",eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yIGRlIGZyb2...,13 hours ago,Full-time,Spain,NaN,NaN,2023-08-02
4,Desarrolladores de software,OPTIMUS Sound Innovation,"Girona, Spain",via LinkedIn,OFERTA DE EMPLEO\n\nOptimus “R+D Department” b...,[{'items': ['OFERTA DE EMPLEO Optimus “R+D De...,[{'link': 'https://www.google.com/search?sca_e...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[4 days ago, Full-time, No degree mentioned]",eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yZXMgZGUgc2...,4 days ago,Full-time,Spain,NaN,NaN,2023-08-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9938,SOPORTE TÉCNICO REMOTO (Energías renovables),SEGULA Technologies - Barcelona,"Barcelona, Spain",via Remotify.es,"Descripción:\n\nGrupo Segula, precisa incorpor...","[{'items': ['Descripción: Grupo Segula, preci...",[{'link': 'https://www.google.com/search?sca_e...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[Full-time, No degree mentioned]",eyJqb2JfdGl0bGUiOiJTT1BPUlRFIFTDiUNOSUNPIFJFTU...,NaN,Full-time,Spain,NaN,NaN,2023-08-02
9939,Soporte técnico helpdesk con PORTUGUES (noches),SOSmatic,Anywhere,via Indeed,Buscamos un perfil Técnico/a Helpdesk de Sopor...,[{'items': ['Buscamos un perfil Técnico/a Help...,[{'link': 'https://www.google.com/search?sca_e...,NaN,"[12 days ago, €14,000–€15,750 a year, Work fro...",eyJqb2JfdGl0bGUiOiJTb3BvcnRlIHTDqWNuaWNvIGhlbH...,12 days ago,Part-time,Spain,True,NaN,2023-08-02
9940,Soporte tecnico helpdesk con portugues noches,jobzem (16214639),Spain,via Jobrapido.com,cfgm informatica o similar o conocimientos equ...,[{'items': ['cfgm informatica o similar o cono...,[{'link': 'https://www.google.com/search?sca_e...,NaN,"[6 days ago, Full-time, No degree mentioned]",eyJqb2JfdGl0bGUiOiJTb3BvcnRlIHRlY25pY28gaGVscG...,6 days ago,Full-time,Spain,NaN,NaN,2023-08-02
9941,Servicio Técnico para Soporte Pardot (INGLES) ...,Krell Consulting,Anywhere,via Indeed,hace 3 días\n\nDescripción\n\nEn Krell Consult...,[{'items': ['hace 3 días Descripción En Krel...,[{'link': 'https://www.google.com/search?sca_e...,NaN,"[4 days ago, €29K a year, Work from home, Full...",eyJqb2JfdGl0bGUiOiJTZXJ2aWNpbyBUw6ljbmljbyBwYX...,4 days ago,Full-time,Spain,True,NaN,2023-08-02


In [12]:
%%time

job_types = ["Full-time", "Part-time", "Contractor", "Internship"]

# Drops Duplicates
df_new = df_new.drop_duplicates("job_id").reset_index(drop = True) 

# Cleans "via"
df_new["via"] = df_new["via"].apply(lambda x : clean.clean_source(x) if not pd.isna(x) else x)

# Cleans "location"
df_new["location"] = df_new["location"].apply(lambda x : clean.clean_location(x))

# Cleans "contract_type"
# df_new["schedule_type"] = df_new["schedule_type"].apply(lambda x : clean.clean_contract_type(x) if x not in job_types else x)

# Cleans "created_date"
df_new["posted_at"] = df_new["posted_at"].apply(lambda x : clean.transform_date(x))

df_new["date_posted"] = [clean.get_date(x, y) for x, y in df_new[["date_posted", "posted_at"]].values]

# Creates "tech_skills"
df_new["tech_skills"] = df_new["description"].apply(lambda x : clean.get_skills(x) if not pd.isna(x) else x)

CPU times: total: 24.8 s
Wall time: 38.6 s


In [13]:
df_new.to_csv("españa03092023_español.csv", index = False)